In [1]:
import datetime
today = datetime.date.today()
yesterday = today - datetime.timedelta(1)  
date = yesterday.strftime('%Y%m%d')

print(date)

20200826


# 1. 영화랭킹(평점순)에서 장르별 30개씩 스크래핑하기

In [63]:
# 랭킹 1~50위까지 영화제목, 상세페이지 크롤링
import requests
from bs4 import BeautifulSoup
import bs4
from urllib.parse import urljoin
import re

main_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date='+ date

movie_list = []
for i in [1,2,4,5,6,7,11,13,15,16,18,19]:
    
    for page_num in [1,2]:
        movie_genre_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date='+str(date)+'&tg='+str(i)+'&page='+str(page_num)
        html = requests.get(movie_genre_url).text
        soup = BeautifulSoup(html, 'html.parser')

        a_tag_list = soup.select('div .tit5 a[href*=basic.nhn]')


        for idx,a_tag in enumerate(a_tag_list,1):
            if idx > 50:
                break
            movie_dict = {}
            title = a_tag.text.strip()
            link = urljoin(main_url, a_tag['href'])

            movie_dict['genre_code'] = i
            movie_dict['id'] = idx
            movie_dict['title'] = title
            movie_dict['url'] = link
            # scorre_url 은 평점을 스크래핑하기 위한 페이지
            movie_dict['score_url'] = link.replace('basic', 'pointWriteFormList')
            movie_list.append(movie_dict)
            #print(idx, title, link)

print(len(movie_list))        
movie_list


1187


[{'genre_code': 1,
  'id': 1,
  'title': '그린 북',
  'url': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=171539',
  'score_url': 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=171539'},
 {'genre_code': 1,
  'id': 2,
  'title': '가버나움',
  'url': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=174830',
  'score_url': 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=174830'},
 {'genre_code': 1,
  'id': 3,
  'title': '원더',
  'url': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=151196',
  'score_url': 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=151196'},
 {'genre_code': 1,
  'id': 4,
  'title': '아일라',
  'url': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=169240',
  'score_url': 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=169240'},
 {'genre_code': 1,
  'id': 5,
  'title': '당갈',
  'url': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=157243',
  'score_url': 'https://movie.naver.com/movie/bi/m

In [64]:
movie_code_list = []
for i in [1,2,4,5,6,7,11,13,15,16,18,19]:
    for page_num in [1,2]:
        movie_genre_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date='+str(date)+'&tg='+str(i)+'&page='+str(page_num)
        html = requests.get(movie_genre_url).text
        soup = BeautifulSoup(html, 'html.parser')

        a_tag_list = soup.select('div .tit5 a[href*=basic.nhn]')

        for idx,a_tag in enumerate(a_tag_list,1):
            if idx > 50:
                break
            movie_code = re.findall("\d+", a_tag['href'])
            
            
            for movie in movie_code:
                movie_code_list.append(movie)

print(len(movie_code_list))
movie_code_list

1187


['171539',
 '174830',
 '151196',
 '169240',
 '157243',
 '17421',
 '154667',
 '156464',
 '82432',
 '22126',
 '17159',
 '106360',
 '10048',
 '34324',
 '134899',
 '31162',
 '161850',
 '147092',
 '14450',
 '19079',
 '12482',
 '152655',
 '17541',
 '142632',
 '51172',
 '10114',
 '73476',
 '189111',
 '34227',
 '185275',
 '31013',
 '35187',
 '103535',
 '10001',
 '14448',
 '71081',
 '39584',
 '169561',
 '10021',
 '12297',
 '17869',
 '167787',
 '70651',
 '29571',
 '150376',
 '151752',
 '79762',
 '18817',
 '51083',
 '70773',
 '16531',
 '15899',
 '75413',
 '100654',
 '146504',
 '29953',
 '41375',
 '30306',
 '29059',
 '16218',
 '10185',
 '83893',
 '71932',
 '10020',
 '50672',
 '34010',
 '36718',
 '63403',
 '158191',
 '39436',
 '68031',
 '31148',
 '142305',
 '144100',
 '10314',
 '85825',
 '38091',
 '10647',
 '172174',
 '143469',
 '10058',
 '22082',
 '134841',
 '34093',
 '154379',
 '139613',
 '10688',
 '75378',
 '134772',
 '31827',
 '168050',
 '152160',
 '19335',
 '154437',
 '63156',
 '173692',
 '398

In [65]:
movie_info_lists = []

for movie in movie_list:
    html = requests.get(movie['url']).text
    soup = BeautifulSoup(html,'html.parser')
    
    movie_info_dict = {}
    
    # 타이틀
    title = soup.select_one('h3.h_movie a[href*=basic.nhn]')
    if title == None:
        movie_info_dict['title'] = None
    else:
        movie_info_dict['title'] = title.text.strip().replace(',', '')

        
    # 네티즌 평점
    score = soup.select_one('div.netizen_score .star_score em')
    if score == None:
        movie_info_dict['score'] = None
    else:
        movie_info_dict['score'] = score.text.strip()
    
    
    # 개봉국가
    nation = soup.select_one('dl.info_spec span a[href*=nation]')
    if nation == None:
        movie_info_dict['nation'] = None
    else:
        movie_info_dict['nation'] = nation.text.strip()
    
    
    # 감독
    director = soup.select_one('dl.info_spec a[href*=code]')
    if director == None:
        movie_info_dict['director'] = None
    else:
        movie_info_dict['director'] = director.text.strip()
    
    
    # 누적관객수 
    audience = soup.select_one('p.count')
    if audience == None:
        movie_info_dict['audience'] = None
    else:
        movie_info_dict['audience'] = re.sub(r"(명)\([^()]*\)", "", audience.text.strip()).replace(',', '')
    
    
    # 관람객 평점 건수
    comment_count = soup.select_one('div.netizen_score .user_count em')
    if comment_count == None:
        movie_info_dict['comment_count'] = None
    else:
        movie_info_dict['comment_count'] = comment_count.text.strip().replace(',', '')
    
    
    # 개봉일자
    release_year = soup.select_one('dl.info_spec a[href*=open]')
    if release_year == None:
        movie_info_dict['release_year'] = None
    else:
        movie_info_dict['release_year'] = release_year.text.strip()
    
    
    # 평점
    movie_info_dict['score_sum'] = 0

    
    # 줄거리
    plot = soup.select_one('.con_tx')
    if plot == None:
        movie_info_dict['plot'] = None
    else: 
        movie_plot = " ".join(re.sub('|  |\t|\r|\n|', '', plot.text).split())
        movie_info_dict['plot'] = movie_plot.replace(',',"")

    movie_info_lists.append(movie_info_dict)

print(len(movie_info_lists))
print(movie_info_lists)

1187
[{'title': '그린 북', 'score': '9.59', 'nation': '미국', 'director': '피터 패럴리', 'audience': None, 'comment_count': '7117', 'release_year': '2019', 'score_sum': 0, 'plot': '1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가(비고 모텐슨)는 교양과 우아함 그 자체인 천재 피아니스트 돈 셜리(마허샬라 알리) 박사의 운전기사 면접을 보게 된다. 백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는 위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고 투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다. 거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사. 생각 행동 말투 취향까지 달라도 너무 다른 두 사람은 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…'}, {'title': '가버나움', 'score': '9.59', 'nation': '레바논', 'director': '나딘 라바키', 'audience': '146605', 'comment_count': '3250', 'release_year': '2019', 'score_sum': 0, 'plot': '나를 세상에 태어나게 한 "부모님을 고소하고 싶어요..." -출생기록조차 없이 살아온 어쩌면 12살 소년 \'자인\'으로부터'}, {'title': '원더', 'score': '9.50', 'nation': '미국', 'director': '스티븐 크보스키', 'audience': None, 'comment_count': '3330', 'release_year': '2017', 'score_sum': 0, 'plot': "누구보다 위트 있고 호기심 많은 매력 부자 ‘어기'(제이콥 트렘블레이). 하지만 남들과 다른 외모로 태어난 ‘어기'는 모두가 좋아하는 크리스마스 대신

## 2.1. movieapp_movie 테이블

In [67]:
import numpy as np
import pandas as pd

data_df = pd.DataFrame(columns=['title', 'score', 'nation', 'director', 'audience', 'plot', 'score_sum', 'comment_count', 'release_year'])

for data in movie_info_lists:
    series_obj = pd.Series(data)
    data_df = data_df.append(series_obj, ignore_index=True)

# 인덱스를 1부터 시작하는 방법
data_df.index = np.arange(1,len(data_df)+1)

data_df

,title,score,nation,director,audience,plot,score_sum,comment_count,release_year
1,그린 북,9.59,미국,피터 패럴리,None,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가(비고 모텐슨)는 교양과 ...,0,7117,2019
2,가버나움,9.59,레바논,나딘 라바키,146605,"나를 세상에 태어나게 한 ""부모님을 고소하고 싶어요..."" -출생기록조차 없이 살아...",0,3250,2019
3,원더,9.50,미국,스티븐 크보스키,None,누구보다 위트 있고 호기심 많은 매력 부자 ‘어기'(제이콥 트렘블레이). 하지만 남...,0,3330,2017
4,아일라,9.49,한국,잔 울카이,None,1950년 한국전쟁에 파병된 ‘슐레이만’은 칠흑 같은 어둠 속 홀로 남겨진 5살 소...,0,1866,2018
5,당갈,9.48,인도,니테쉬 티와리,None,전직 레슬링 선수였던 ‘마하비르 싱 포갓(아미르 칸)’은 아버지의 반대로 금메달의 ...,0,2235,2018
6,쇼생크 탈출,9.44,미국,프랭크 다라본트,None,촉망 받던 은행 부지점장 ‘앤디(팀 로빈슨 分)’는 아내와 그 애인을 살해한 혐의로...,0,18080,2016
7,덕구,9.42,한국,방수인,None,어린 손자와 살고 있는 일흔 살 덕구 할배는 자신에게 주어진 시간이 얼마 남지 않음...,0,2555,2018
8,보헤미안 랩소디,9.42,미국,브라이언 싱어,None,공항에서 수하물 노동자로 일하며 음악의 꿈을 키우던 이민자 출신의 아웃사이더 ‘파록...,0,39199,2018
9,헬프,9.42,미국,테이트 테일러,None,1963년 미국 남부 미시시피 잭슨 흑인 가정부는 백인 주인과 화장실도 같이 쓸 수...,0,3714,2011
10,인생은 아름다워,9.40,이탈리아,로베르토 베니니,None,로마에 갓 상경한 시골 총각 ‘귀도’는 운명처럼 만난 여인 ‘도라’에게 첫눈에 반한...,0,12175,2016


## 2.2. movieapp_genre 테이블

In [68]:
movie_genre_lists = []
for movie in movie_list:
    movie_genre_dict = {}
    
    html = requests.get(movie['url']).text
    soup = BeautifulSoup(html,'html.parser')
    
    title = soup.select_one('h3.h_movie a[href*=basic.nhn]')
    if title == None:
        movie_genre_dict['title'] = None
    else:
        movie_genre_dict['title'] = soup.select_one('h3.h_movie a[href*=basic.nhn]').text.strip().replace(',', '')
        
        
    genre = soup.select_one('dl.info_spec span')
    if genre == None:
        movie_genre_dict['genre'] = None
    else:
        movie_genre_dict['genre'] = re.sub(r"\s+", "", genre.text.strip())
    
    movie_genre_lists.append(movie_genre_dict)
    
    
print(len(movie_genre_lists))

for movie_genre in movie_genre_lists:
    print(movie_genre)

1187
{'title': '그린 북', 'genre': '드라마'}
{'title': '가버나움', 'genre': '드라마'}
{'title': '원더', 'genre': '드라마'}
{'title': '아일라', 'genre': '드라마,전쟁'}
{'title': '당갈', 'genre': '드라마,액션'}
{'title': '쇼생크 탈출', 'genre': '드라마'}
{'title': '덕구', 'genre': '드라마'}
{'title': '보헤미안 랩소디', 'genre': '드라마'}
{'title': '헬프', 'genre': '드라마'}
{'title': '인생은 아름다워', 'genre': '드라마,코미디'}
{'title': '포레스트 검프', 'genre': '드라마,코미디'}
{'title': '위대한 쇼맨', 'genre': '드라마,뮤지컬'}
{'title': '죽은 시인의 사회', 'genre': '드라마'}
{'title': '집으로...', 'genre': '드라마,가족'}
{'title': '동주', 'genre': '드라마'}
{'title': '캐스트 어웨이', 'genre': '드라마,모험'}
{'title': '아이 캔 스피크', 'genre': '드라마'}
{'title': '히든 피겨스', 'genre': '드라마'}
{'title': '쉰들러 리스트', 'genre': '드라마,전쟁'}
{'title': '굿 윌 헌팅', 'genre': '드라마'}
{'title': '여인의 향기', 'genre': '드라마,코미디'}
{'title': '달링', 'genre': '드라마,멜로/로맨스'}
{'title': '굿바이 마이 프랜드', 'genre': '드라마'}
{'title': '미스 슬로운', 'genre': '드라마,스릴러'}
{'title': '지상의 별처럼', 'genre': '드라마'}
{'title': '아마데우스', 'genre': '드라마'}
{'title': '그대를 사랑합니다', 'genre': 

{'title': None, 'genre': None}
{'title': '스타워즈 에피소드 4 - 새로운 희망', 'genre': 'SF,모험,액션,판타지'}
{'title': '스타워즈 에피소드 6 - 제다이의 귀환', 'genre': 'SF,모험,액션,판타지'}
{'title': '디스트릭트 9', 'genre': 'SF,액션,스릴러,드라마'}
{'title': '아이언맨', 'genre': 'SF,액션,드라마,판타지'}
{'title': '엑스맨: 데이즈 오브 퓨처 패스트', 'genre': 'SF,모험,액션,판타지'}
{'title': '아이언맨 3', 'genre': 'SF,모험,액션'}
{'title': '토탈 리콜', 'genre': 'SF,액션'}
{'title': '트랜스포머', 'genre': 'SF,액션,모험'}
{'title': '지구를 지켜라!', 'genre': 'SF,스릴러'}
{'title': '매트릭스 2 - 리로디드', 'genre': 'SF,모험,액션,판타지'}
{'title': '12 몽키즈', 'genre': 'SF,드라마,스릴러'}
{'title': '아일랜드', 'genre': 'SF,모험,액션'}
{'title': '아이 로봇', 'genre': 'SF,스릴러,모험,액션'}
{'title': '블레이드 러너', 'genre': 'SF,액션,드라마'}
{'title': '공각기동대', 'genre': 'SF,판타지,애니메이션'}
{'title': '딥 임팩트', 'genre': 'SF,드라마'}
{'title': '13 구역', 'genre': 'SF,액션'}
{'title': '스타 트렉: 더 비기닝', 'genre': 'SF,모험,액션'}
{'title': '맨 인 블랙', 'genre': 'SF,액션,코미디'}
{'title': '레지던트 이블', 'genre': 'SF,스릴러,모험,액션,공포'}
{'title': '매트릭스 3 - 레볼루션', 'genre': 'SF,스릴러,액션'}
{'title': '더 문',

In [69]:
final_genre_list = []

for idx, movie_genre in enumerate(movie_genre_lists, 1):
    if movie_genre['genre'] == None:
        movie_genre['genre'] = '로그인 필요(19금)'
    else:
        genres = movie_genre['genre'].split(',')
    
    for genre in genres:
        genre_dict = {}
        genre_dict['movie_id'] = idx
        genre_dict['genre'] = genre
        final_genre_list.append(genre_dict)
        
final_genre_list

[{'movie_id': 1, 'genre': '드라마'},
 {'movie_id': 2, 'genre': '드라마'},
 {'movie_id': 3, 'genre': '드라마'},
 {'movie_id': 4, 'genre': '드라마'},
 {'movie_id': 4, 'genre': '전쟁'},
 {'movie_id': 5, 'genre': '드라마'},
 {'movie_id': 5, 'genre': '액션'},
 {'movie_id': 6, 'genre': '드라마'},
 {'movie_id': 7, 'genre': '드라마'},
 {'movie_id': 8, 'genre': '드라마'},
 {'movie_id': 9, 'genre': '드라마'},
 {'movie_id': 10, 'genre': '드라마'},
 {'movie_id': 10, 'genre': '코미디'},
 {'movie_id': 11, 'genre': '드라마'},
 {'movie_id': 11, 'genre': '코미디'},
 {'movie_id': 12, 'genre': '드라마'},
 {'movie_id': 12, 'genre': '뮤지컬'},
 {'movie_id': 13, 'genre': '드라마'},
 {'movie_id': 14, 'genre': '드라마'},
 {'movie_id': 14, 'genre': '가족'},
 {'movie_id': 15, 'genre': '드라마'},
 {'movie_id': 16, 'genre': '드라마'},
 {'movie_id': 16, 'genre': '모험'},
 {'movie_id': 17, 'genre': '드라마'},
 {'movie_id': 18, 'genre': '드라마'},
 {'movie_id': 19, 'genre': '드라마'},
 {'movie_id': 19, 'genre': '전쟁'},
 {'movie_id': 20, 'genre': '드라마'},
 {'movie_id': 21, 'genre': '드라마'},
 

In [70]:
import numpy as np
import pandas as pd

genre_data_df = pd.DataFrame(columns = ['genre', 'movie_id'])
for final_genre in final_genre_list:
    series_obj = pd.Series(final_genre)
    genre_data_df = genre_data_df.append(series_obj, ignore_index=True)
    
# 인덱스를 1부터 시작하는 방법
genre_data_df.index = np.arange(1,len(genre_data_df)+1)
genre_data_df

,genre,movie_id
1,드라마,1
2,드라마,2
3,드라마,3
4,드라마,4
5,전쟁,4
6,드라마,5
7,액션,5
8,드라마,6
9,드라마,7
10,드라마,8


### 2.3. movieapp_actor 테이블

In [72]:
movie_actor_list = []
for movie in movie_list:
    movie_actor_dict = {}
    #print(movie)
    html = requests.get(movie['url']).text
    soup = BeautifulSoup(html,'html.parser')
  
    actor = soup.find('dl',attrs={"class":"info_spec"})
    if actor == None:
        movie_actor_dict['actor'] = None
    else:
        movie_actor_dict['actor'] = re.sub(r'\([^)]*\)', '', actor.find_all("dd")[2].text.replace('더보기', ''))
    
    movie_actor_list.append(movie_actor_dict)
    
    
print(len(movie_actor_list))

for movie_actor in movie_actor_list:
    print(movie_actor)

1187
{'actor': '비고 모텐슨, 마허샬라 알리'}
{'actor': '자인 알 라피아, 요르다노스 시프로우'}
{'actor': '제이콥 트렘블레이, 줄리아 로버츠, 오웬 윌슨'}
{'actor': '김설, 이스마일 하지오글루'}
{'actor': '아미르 칸, 파티마 사나 셰이크, 산야 말호트라'}
{'actor': '팀 로빈스, 모건 프리먼'}
{'actor': '이순재, 정지훈'}
{'actor': '라미 말렉, 루시 보인턴, 귈림 리'}
{'actor': '엠마 스톤, 비올라 데이비스, 옥타비아 스펜서'}
{'actor': '로베르토 베니니, 니콜레타 브라스키'}
{'actor': '톰 행크스'}
{'actor': '휴 잭맨, 잭 에프론, 미셸 윌리엄스'}
{'actor': '로빈 윌리엄스'}
{'actor': '김을분, 유승호'}
{'actor': '강하늘, 박정민, 김인우'}
{'actor': '톰 행크스, 헬렌 헌트'}
{'actor': '나문희, 이제훈'}
{'actor': '타라지 P. 헨슨, 옥타비아 스펜서, 자넬 모네'}
{'actor': '리암 니슨'}
{'actor': '맷 데이먼'}
{'actor': '알 파치노'}
{'actor': '앤드류 가필드, 클레어 포이'}
{'actor': '조셉 마젤로, 브래드 렌프로, 다이아나 스카위드'}
{'actor': '제시카 차스테인'}
{'actor': '다쉴 사페리, 아미르 칸'}
{'actor': '톰 헐스, F. 머레이 아브라함, 엘리자베스 베리지'}
{'actor': '이순재, 윤소정, 송재호'}
{'actor': '안소니 홉킨스, 조나단 프라이스'}
{'actor': '숀 펜, 미셸 파이퍼'}
{'actor': '크리스 히친, 데비 허니우드'}
{'actor': '제이미 벨, 줄리 월터스, 게리 루이스'}
{'actor': '애드리언 브로디, 토마스 크레취만'}
{'actor': '설경구, 엄지원, 이레'}
{'actor': '마르코 레오나르디, 필립 느와레, 자끄 페렝'}


{'actor': '마츠오카 요시츠구, 토마츠 하루카, 이토 카나에'}
{'actor': '오오타니 이쿠에'}
{'actor': '타케우치 준코, 나카무라 치에, 모리카와 토시유키'}
{'actor': '송강호, 김상경'}
{'actor': '장 르노, 나탈리 포트만'}
{'actor': '데니스 퀘이드, 제임스 카비젤'}
{'actor': '클린트 이스트우드, 크리스토퍼 칼리, 비 방'}
{'actor': '알 파치노, 다이안 키튼, 탈리아 샤이어'}
{'actor': '알 파치노, 숀 펜, 페네로프 앤 밀러'}
{'actor': '설경구, 이성재'}
{'actor': '알 파치노, 로버트 듀발, 다이안 키튼'}
{'actor': '레오나르도 디카프리오, 톰 행크스'}
{'actor': None}
{'actor': '톰 행크스, 데이빗 모스, 보니 헌트'}
{'actor': '조승우, 김혜수, 백윤식'}
{'actor': '톰 크루즈, 잭 니콜슨, 데미 무어'}
{'actor': '케빈 코스트너'}
{'actor': '말론 브란도, 알 파치노, 제임스 칸'}
{'actor': '양조위, 유덕화'}
{'actor': '알 파치노, 로버트 드 니로'}
{'actor': '김윤석, 하정우'}
{'actor': '데브 파텔, 아미 해머, 나자닌 보니아디'}
{'actor': '김윤진, 김미숙, 박희순'}
{'actor': None}
{'actor': '이병헌, 조승우, 백윤식'}
{'actor': '조디 포스터, 안소니 홉킨스, 스콧 글렌'}
{'actor': '이선균, 조진웅'}
{'actor': '주윤발, 장국영, 중추훙'}
{'actor': '이병헌, 조승우, 백윤식'}
{'actor': '유덕화, 량차오웨이, 여명'}
{'actor': '마리오 카사스, 바바라 레니, 호세 코로나도'}
{'actor': '이정재, 최민식, 황정민'}
{'actor': '알 파치노'}
{'actor': '리차드 기어'}
{'actor': '이완 맥그리거, 이완 브렘너, 조니 리

In [74]:
final_actor_list = []

for idx, movie_actor in enumerate(movie_actor_list, 1):
    if movie_actor['actor'] == None:
        movie_actor['actor'] = None
    else:
        actors = movie_actor['actor'].split(', ')
    
    for actor in actors:
        actor_dict = {}
        actor_dict['movie_id'] = idx
        actor_dict['actor'] = actor
        final_actor_list.append(actor_dict)
        
final_actor_list

[{'movie_id': 1, 'actor': '비고 모텐슨'},
 {'movie_id': 1, 'actor': '마허샬라 알리'},
 {'movie_id': 2, 'actor': '자인 알 라피아'},
 {'movie_id': 2, 'actor': '요르다노스 시프로우'},
 {'movie_id': 3, 'actor': '제이콥 트렘블레이'},
 {'movie_id': 3, 'actor': '줄리아 로버츠'},
 {'movie_id': 3, 'actor': '오웬 윌슨'},
 {'movie_id': 4, 'actor': '김설'},
 {'movie_id': 4, 'actor': '이스마일 하지오글루'},
 {'movie_id': 5, 'actor': '아미르 칸'},
 {'movie_id': 5, 'actor': '파티마 사나 셰이크'},
 {'movie_id': 5, 'actor': '산야 말호트라'},
 {'movie_id': 6, 'actor': '팀 로빈스'},
 {'movie_id': 6, 'actor': '모건 프리먼'},
 {'movie_id': 7, 'actor': '이순재'},
 {'movie_id': 7, 'actor': '정지훈'},
 {'movie_id': 8, 'actor': '라미 말렉'},
 {'movie_id': 8, 'actor': '루시 보인턴'},
 {'movie_id': 8, 'actor': '귈림 리'},
 {'movie_id': 9, 'actor': '엠마 스톤'},
 {'movie_id': 9, 'actor': '비올라 데이비스'},
 {'movie_id': 9, 'actor': '옥타비아 스펜서'},
 {'movie_id': 10, 'actor': '로베르토 베니니'},
 {'movie_id': 10, 'actor': '니콜레타 브라스키'},
 {'movie_id': 11, 'actor': '톰 행크스'},
 {'movie_id': 12, 'actor': '휴 잭맨'},
 {'movie_id': 12, 'actor'

In [75]:
import numpy as np
import pandas as pd

actor_data_df = pd.DataFrame(columns = ['actor', 'movie_id'])
for final_actor in final_actor_list:
    series_obj = pd.Series(final_actor)
    actor_data_df = actor_data_df.append(series_obj, ignore_index=True)
    
# 인덱스를 1부터 시작하는 방법
# 인덱스 값 변겅
actor_data_df.index = np.arange(1,len(actor_data_df)+1)
actor_data_df

,actor,movie_id
1,비고 모텐슨,1
2,마허샬라 알리,1
3,자인 알 라피아,2
4,요르다노스 시프로우,2
5,제이콥 트렘블레이,3
6,줄리아 로버츠,3
7,오웬 윌슨,3
8,김설,4
9,이스마일 하지오글루,4
10,아미르 칸,5


## 2.3. movieapp_comment 테이블 (미완성)

In [76]:
comment_list = []
data = datetime.datetime.now()
data.strftime('%Y-%m-%d %H:%M:%S.%f')

for idx, movie_code in enumerate(movie_code_list, 1):   
    comment_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='+ movie_code +'&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
    
    for i in range(1,11):
        comment_detail_urls = comment_url + str(i)
        html = requests.get(comment_detail_urls).text
        soup = BeautifulSoup(html, 'html.parser')
        comment_tag_list = soup.select('span[id^=_filtered_ment_]')
        #print(comment_tag_list)

        for comment_tag in comment_tag_list:
            space_comment = comment_tag.text.strip()
            no_space_comment = re.sub('|  |\t|\n', '', space_comment)
            #print(no_space_comment)

            comment_dict = {}
            comment_dict['movie_id'] = idx
            comment_dict['comment'] = no_space_comment.replace(',','')
            comment_dict['published_date'] = data.strftime('%Y-%m-%d %H:%M:%S.%f')
            comment_dict['comment_score'] = 0
            comment_dict['user_id'] = 0
            comment_list.append(comment_dict)
            
            if len(comment_tag_list) != 10:
                break;
    
comment_list

[{'movie_id': 1,
  'comment': '폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행동을 예측했을 것이다. 하지만 그 예측 자체가 영화보는내내 우리에게 생긴 편견임을 알고 뒤통수를 맞은 느낌이었다!',
  'published_date': '2020-08-27 16:30:00.468905',
  'comment_score': 0,
  'user_id': 0},
 {'movie_id': 1,
  'comment': "우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무려놓아 눈물 쥐어짠 걸 의미있는 영화라 자아도취해 있을 동안 헐리우드는 실화 소재로 이런 '작품'을 배출하는 경지에 ...",
  'published_date': '2020-08-27 16:30:00.468905',
  'comment_score': 0,
  'user_id': 0},
 {'movie_id': 1,
  'comment': '하.. 이런 영화가 상영관수 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려한 거품영화만 너무 독차지 하고있으니..아쉽',
  'published_date': '2020-08-27 16:30:00.468905',
  'comment_score': 0,
  'user_id': 0},
 {'movie_id': 1,
  'comment': '보고나면 KFC 치킨이 존나 떙기는 영화임..',
  'published_date': '2020-08-27 16:30:00.468905',
  'comment_score': 0,
  'user_id': 0},
 {'movie_id': 1,
  'comment': '빗속 셜리의 대사는 소름이 끼쳤다',
  'published_date': '2020-08-27 16:30:00.468905',
  'comment_score': 0,
  'user_id': 0},
 {'movie_id': 1,
  'comment': '정말 많은 여운이 남은 영화네요. 

In [78]:
import numpy as np
import pandas as pd

comment_data_df = pd.DataFrame(columns = ['comment', 'published_date', 'movie_id', 'comment_score', 'user_id'])
for comment in comment_list:
    series_obj = pd.Series(comment)
    comment_data_df = comment_data_df.append(series_obj, ignore_index=True)
    
# 인덱스를 1부터 시작하는 방법
# 인덱스 값 변겅
comment_data_df.index = np.arange(1,len(comment_data_df)+1)
comment_data_df

,comment,published_date,movie_id,comment_score,user_id
1,폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행...,2020-08-27 16:30:00.468905,1,0,0
2,우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무...,2020-08-27 16:30:00.468905,1,0,0
3,하.. 이런 영화가 상영관수 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려한...,2020-08-27 16:30:00.468905,1,0,0
4,보고나면 KFC 치킨이 존나 떙기는 영화임..,2020-08-27 16:30:00.468905,1,0,0
5,빗속 셜리의 대사는 소름이 끼쳤다,2020-08-27 16:30:00.468905,1,0,0
6,정말 많은 여운이 남은 영화네요. 꼭 봐야 할영화입니다,2020-08-27 16:30:00.468905,1,0,0
7,비고 모텐슨 형님 반지의 제왕에서 꽃미남 왕으로 열연하던게 엊그제 같은데 중년 마초...,2020-08-27 16:30:00.468905,1,0,0
8,2019년 아카데미 작품상은 그린 북 남우조연상은 마허샬라 알리!,2020-08-27 16:30:00.468905,1,0,0
9,경찰관에게 메리크리스마스 하는 장면..너무 조하 따뜻해..,2020-08-27 16:30:00.468905,1,0,0
10,고상함 뒤에 숨겨진 추악함이란.토니의 변화는 놀랍고 셜리의 용기는 존경스럽다.금년 ...,2020-08-27 16:30:00.468905,1,0,0


# 3. 판다스 데이터프레임 CSV 파일로 export하기

In [79]:
data_df.to_csv('C:/mypython/project_test/csv/movieapp_movie.csv', index=True, header=False, encoding='utf-8-sig')
genre_data_df.to_csv('C:/mypython/project_test/csv/movieapp_genre.csv', index=True, header=False, encoding='utf-8-sig')
actor_data_df.to_csv('C:/mypython/project_test/csv/movieapp_actor.csv', index=True, header=False, encoding='utf-8-sig')
comment_data_df.to_csv('C:/mypython/project_test/csv/movieapp_comment.csv', index=True, header=False, encoding='utf-8-sig')